## 朴素贝叶斯 实现中文文本分类

数据说明：文本二分类，1和0，判断是否属于政治上的出访类事件

数据来源：[https://github.com/ares5221/ALBERT_text_classification](https://github.com/ares5221/ALBERT_text_classification)

目录

* [1.加载数据](#1.加载数据)
* [2.文本预处理（清洗文本，分词，去除停用词）](#2.文本预处理（清洗文本，分词，去除停用词）)
* [3.抽取文本特征：使用词袋模型](#3.抽取文本特征：使用词袋模型)
* [4.训练贝叶斯模型（多项式贝叶斯）](#4.训练贝叶斯模型（多项式贝叶斯）)
* [5.评价指标&测试](#5.评价指标&测试)
* [6.交叉验证](#6.交叉验证)

## 1.加载数据

In [14]:
import pandas as pd
import re
import jieba
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
import os
# print(os.getcwd())

project_dir = '/home/xijian/pycharm_projects/Magic-NLPer/'
# 当前目录cwd
cwd = project_dir + 'MachineLearning/NaiveBayes朴素贝叶斯/'
data_dir = cwd + './data/zzcf/' # 数据所在目录
stopwords_file = project_dir + 'zh_data/stopwords.txt'

In [60]:
df_train = pd.read_csv(data_dir+'train.txt', encoding='UTF-8', sep='\s', header=None,
                       names=['label', 'content'], index_col=False)
df_train = df_train.dropna() # 过滤含有NaN的数据

df_test = pd.read_csv(data_dir+'test.txt', encoding='UTF-8', sep='\s', header=None,
                       names=['label', 'content'], index_col=False)
df_test = df_test.dropna() # 过滤含有NaN的数据

print(df_train.head())
print()
print(df_train['label'].value_counts())

   label                                            content
0      1  当地时间2月10日，白宫发表声明称，美国总统特朗普及夫人梅拉尼娅将于2月24日至25日访问印...
1      0  俄罗斯卫星通讯社11日最新消息，菲律宾总统杜特尔特已下令终止与美国间的《访问部队协定》(VFA)。
2      1  据俄罗斯卫星网6日报道，土耳其总统发言人卡林表示，俄罗斯军事代表团将于近日访问安卡拉，讨论叙...
3      0  先来说说什么是LPDDR5：要知道，手机中有两种内存颗粒，一种就是DRAM也就是大家常说的“...
4      1  在疫情的关键时刻，出现了一件令人感动的事情，让我们明白这才是真正的好朋友，不惧疫情访问我国，...

0    151
1    149
Name: label, dtype: int64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [61]:
print(df_train.shape, df_test.shape) # (300, 2), (80, 2)

(300, 2) (80, 2)


## 2.文本预处理（清洗文本，分词，去除停用词）

In [62]:
# 保留文本中文、数字、英文、短横线
def clear_text(text):
    p = re.compile(r"[^\u4e00-\u9fa5^0-9^a-z^A-Z\-、，。！？：；（）《》【】,!\?:;[\]()]")  # 匹配不是中文、数字、字母、短横线的部分字符
    return p.sub('', text)  # 将text中匹配到的字符替换成空字符

# 加载停用词表
def load_stopwords_file(filename):
    print('加载停用词...')
    stopwords=pd.read_csv(filename, index_col=False, quoting=3, sep="\t", names=['stopword'], encoding='utf-8')
    #quoting：控制引用字符引用行为，QUOTE_MINIMAL (0), QUOTE_ALL (1), QUOTE_NONNUMERIC (2) or QUOTE_NONE (3).
    stopwords = set(stopwords['stopword'].values)
    print('停用词表大小：', len(stopwords))
    return stopwords


# 文本预处理：清洗，分词，并去除停用词
def preprocess_text(df_content):
    stopwords_set = load_stopwords_file(stopwords_file) # 加载停用词表
    content_seg = []#分词后的content
    for i,text in enumerate(df_content):
        text = clear_text(text.strip())
        segs = jieba.lcut(text, cut_all=False)  # cut_all=False是精确模式，True是全模式；默认模式是False 返回分词后的列表
        segs = filter(lambda x: len(x.strip())>1, segs)  # 词长度要>1，没有保留标点符号
        segs = filter(lambda x: x not in stopwords_set, segs)
        # print(segs) # segs是一个filter object
        # segs = list(segs) # segs需要一次类似“持久化”的操作，否则每次被操作一次后segs就为空了
        content_seg.append(" ".join(segs))
    return content_seg

df_train['content_seg'] = preprocess_text(df_train['content'])
df_test['content_seg'] = preprocess_text(df_test['content'])
print(df_train.head())  #
print(df_train.shape)

加载停用词...
停用词表大小： 2613
加载停用词...
停用词表大小： 2613
   label                                            content  \
0      1  当地时间2月10日，白宫发表声明称，美国总统特朗普及夫人梅拉尼娅将于2月24日至25日访问印...   
1      0  俄罗斯卫星通讯社11日最新消息，菲律宾总统杜特尔特已下令终止与美国间的《访问部队协定》(VFA)。   
2      1  据俄罗斯卫星网6日报道，土耳其总统发言人卡林表示，俄罗斯军事代表团将于近日访问安卡拉，讨论叙...   
3      0  先来说说什么是LPDDR5：要知道，手机中有两种内存颗粒，一种就是DRAM也就是大家常说的“...   
4      1  在疫情的关键时刻，出现了一件令人感动的事情，让我们明白这才是真正的好朋友，不惧疫情访问我国，...   

                                         content_seg  
0  时间 白宫 发表声明 美国 总统 特朗普 夫人 拉尼 日至 访问 印度 特朗普 上任 首次 ...  
1  俄罗斯 卫星 通讯社 最新消息 菲律宾 总统 杜特 尔特 下令 终止 美国 访问 部队 协定...  
2  俄罗斯 卫星 报道 土耳其 总统 发言人 卡林 俄罗斯 军事 代表团 近日 访问 安卡拉 讨...  
3  LPDDR5 手机 中有 两种 内存 颗粒 一种 DRAM 常说 运行 内存 提到 LPDD...  
4      疫情 关键时刻 一件 令人感动 事情 明白 这才 朋友 疫情 访问 我国 王毅 机场 迎接  
(300, 3)


## 3.抽取文本特征：使用词袋模型

In [64]:
vectorizer = CountVectorizer(analyzer='word', # 以词为粒度做ngram
                             max_features=4000, # 保留最常见的4000个词
                             )
vectorizer.fit(df_train['content_seg'].tolist())
print('CountVectorizer train finished!')
train_features = vectorizer.transform(df_train['content_seg'])

CountVectorizer train finished!


## 4.训练贝叶斯模型（多项式贝叶斯）

In [65]:
bayes_classifier = MultinomialNB() # 使用多项式贝叶斯
bayes_classifier.fit(train_features, df_train['label'])
print('MultinomialNB train finished!')

MultinomialNB train finished!


## 5.评价指标&测试

In [66]:
test_features = vectorizer.transform(df_test['content_seg'])
bayes_classifier.score(test_features, df_test['label']) # Return the mean accuracy

0.9375

In [67]:
from sklearn.metrics import precision_score, recall_score, f1_score
y_pred = bayes_classifier.predict(test_features)
print('precision_score: %.3f' % precision_score(y_pred, df_test['label']))
print('recall_score: %.3f' % recall_score(y_pred, df_test['label']))
print('f1_score: %.3f' % f1_score(y_pred, df_test['label']))

precision_score: 0.952
recall_score: 0.930
f1_score: 0.941


## 6.交叉验证

In [57]:
df_data = df_train.append(df_test)
print(df_data.shape)
# print(df_data.head())
df_data = df_data.sample(frac=1.0) # shuffle 打乱顺序
# print(df_data.head())

(380, 3)


In [58]:
from sklearn.model_selection import StratifiedKFold
import numpy as np

def stratifiedkfold_cv(X, y, clf_class, shuffle=True, n_folds=5, **kwargs):
    skfold = StratifiedKFold(n_splits=n_folds, shuffle=shuffle, random_state=0)
    y_pred = y[:]
    for train_idx, test_idx in skfold.split(X, y):
        X_tr, y_tr = X[train_idx], y[train_idx] # 训练集
        X_te, y_te = X[test_idx], y[test_idx] # 测试集
        clf = clf_class(**kwargs)
        clf.fit(X_tr,y_tr)
        y_pred[test_idx] = clf.predict(X_te)
    return y_pred


y_pred = stratifiedkfold_cv(vectorizer.transform(df_data['content_seg']),
                            np.array(df_data['label']),
                            MultinomialNB)

print(precision_score(df_data['label'], y_pred))

0.8719211822660099
